<a id="top"></a>
<div style="text-align: center; background: #ff8c00; font-family: 'Montserrat', sans-serif; color: white; padding: 15px; font-size: 30px; font-weight: bold; line-height: 1; border-radius: 20px 20px 0 0; margin-bottom: 20px; box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.2);">🚀 Indian Actor recognition using Facenet 🚀</div>


<div style="text-align: center; background: #1ED760; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 15px; font-size: 26px; font-weight: bold; line-height: 1; border-radius: 50% 0 50% 0 / 40px; margin-bottom: 20px; box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);">Life Cycle of the Project</div>


<div style="font-size: 14px; font-family: Verdana; border: 2px solid #ccc; background-color: #F5F5F5; padding: 10px; border-radius: 10px; margin-bottom: 20px; position: relative;">

  <!-- Add the image inside the div, aligned to the right-center -->
   <img src="https://media.giphy.com/media/2lQzj96U6fb5qbknOW/giphy.gif" style="position: absolute; left:40%; right: 0; transform: translateY(-50%); width: 200px; height: auto;">
   <img src="https://media.giphy.com/media/2lQzj96U6fb5qbknOW/giphy.gif" style="position: absolute; left:60%; right: 0; transform: translateY(-50%); width: 200px; height: auto;">
    
  <span style="color: #FF5733; font-weight: bold;">Steps to be Performed</span>
  <ol>
    <li><a href="#1">Loading facenet with pretrained weights</a></li>
    <li><a href="#2">Loading required packages</a></li>
    <li><a href="#3">Face Embeddings Genration</a></li>
    <li><a href="#4">Mapping targets to discrete labels</a></li>
    <li><a href="#5">Divinding data into train and test splits</a></li>
    <li><a href="#6">Model Training 🤖</a></li>
    <li><a href="#7">Model Evaluation 📈</a></li>
    <li><a href="#8">Downloading trained SVM model</a></li>
    <li><a href="#9">Model Inferencing</a></li>
    <li><a href="#10">Author Message ✉️</a></li>
  </ol>
</div>


<div style="text-align: center; background: #1ED760; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 15px; font-size: 26px; font-weight: bold; line-height: 1; border-radius: 50% 0 50% 0 / 40px; margin-bottom: 20px; box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);">Problem Statement</div>

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">

The aim of this kernel is to develop a model that is capable of recognizing the trained faces of 135 actors. To accomplish this we are going with the classification of face embeddings generated by Facenet using SVM model.
</div>


<a id="1"></a>
<div style="text-align: center; background: #1ED760; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 15px; font-size: 26px; font-weight: bold; line-height: 1; border-radius: 50% 0 50% 0 / 40px; margin-bottom: 20px; box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);">1. Building Facenet model Architecture and loading weights.</div>

In [ ]:
from functools import partial

from keras.models import Model
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.layers import Concatenate
from keras.layers import Conv2D
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import GlobalAveragePooling2D
from keras.layers import Input
from keras.layers import Lambda
from keras.layers import MaxPooling2D
from keras.layers import add
from keras import backend as K


def scaling(x, scale):
    return x * scale


def conv2d_bn(x,
              filters,
              kernel_size,
              strides=1,
              padding='same',
              activation='relu',
              use_bias=False,
              name=None):
    x = Conv2D(filters,
               kernel_size,
               strides=strides,
               padding=padding,
               use_bias=use_bias,
               name=name)(x)
    if not use_bias:
        bn_axis = 1 if K.image_data_format() == 'channels_first' else 3
        bn_name = _generate_layer_name('BatchNorm', prefix=name)
        x = BatchNormalization(axis=bn_axis, momentum=0.995, epsilon=0.001,
                               scale=False, name=bn_name)(x)
    if activation is not None:
        ac_name = _generate_layer_name('Activation', prefix=name)
        x = Activation(activation, name=ac_name)(x)
    return x


def _generate_layer_name(name, branch_idx=None, prefix=None):
    if prefix is None:
        return None
    if branch_idx is None:
        return '_'.join((prefix, name))
    return '_'.join((prefix, 'Branch', str(branch_idx), name))


def _inception_resnet_block(x, scale, block_type, block_idx, activation='relu'):
    channel_axis = 1 if K.image_data_format() == 'channels_first' else 3
    if block_idx is None:
        prefix = None
    else:
        prefix = '_'.join((block_type, str(block_idx)))
    name_fmt = partial(_generate_layer_name, prefix=prefix)

    if block_type == 'Block35':
        branch_0 = conv2d_bn(x, 32, 1, name=name_fmt('Conv2d_1x1', 0))
        branch_1 = conv2d_bn(x, 32, 1, name=name_fmt('Conv2d_0a_1x1', 1))
        branch_1 = conv2d_bn(branch_1, 32, 3, name=name_fmt('Conv2d_0b_3x3', 1))
        branch_2 = conv2d_bn(x, 32, 1, name=name_fmt('Conv2d_0a_1x1', 2))
        branch_2 = conv2d_bn(branch_2, 32, 3, name=name_fmt('Conv2d_0b_3x3', 2))
        branch_2 = conv2d_bn(branch_2, 32, 3, name=name_fmt('Conv2d_0c_3x3', 2))
        branches = [branch_0, branch_1, branch_2]
    elif block_type == 'Block17':
        branch_0 = conv2d_bn(x, 128, 1, name=name_fmt('Conv2d_1x1', 0))
        branch_1 = conv2d_bn(x, 128, 1, name=name_fmt('Conv2d_0a_1x1', 1))
        branch_1 = conv2d_bn(branch_1, 128, [1, 7], name=name_fmt('Conv2d_0b_1x7', 1))
        branch_1 = conv2d_bn(branch_1, 128, [7, 1], name=name_fmt('Conv2d_0c_7x1', 1))
        branches = [branch_0, branch_1]
    elif block_type == 'Block8':
        branch_0 = conv2d_bn(x, 192, 1, name=name_fmt('Conv2d_1x1', 0))
        branch_1 = conv2d_bn(x, 192, 1, name=name_fmt('Conv2d_0a_1x1', 1))
        branch_1 = conv2d_bn(branch_1, 192, [1, 3], name=name_fmt('Conv2d_0b_1x3', 1))
        branch_1 = conv2d_bn(branch_1, 192, [3, 1], name=name_fmt('Conv2d_0c_3x1', 1))
        branches = [branch_0, branch_1]
    else:
        raise ValueError('Unknown Inception-ResNet block type. '
                         'Expects "Block35", "Block17" or "Block8", '
                         'but got: ' + str(block_type))

    mixed = Concatenate(axis=channel_axis, name=name_fmt('Concatenate'))(branches)
    up = conv2d_bn(mixed,
                   K.int_shape(x)[channel_axis],
                   1,
                   activation=None,
                   use_bias=True,
                   name=name_fmt('Conv2d_1x1'))
    up = Lambda(scaling,
                output_shape=K.int_shape(up)[1:],
                arguments={'scale': scale})(up)
    x = add([x, up])
    if activation is not None:
        x = Activation(activation, name=name_fmt('Activation'))(x)
    return x


def InceptionResNetV1(input_shape=(160, 160, 3),
                      classes=128,
                      dropout_keep_prob=0.8,
                      weights_path=None):
    inputs = Input(shape=input_shape)
    x = conv2d_bn(inputs, 32, 3, strides=2, padding='valid', name='Conv2d_1a_3x3')
    x = conv2d_bn(x, 32, 3, padding='valid', name='Conv2d_2a_3x3')
    x = conv2d_bn(x, 64, 3, name='Conv2d_2b_3x3')
    x = MaxPooling2D(3, strides=2, name='MaxPool_3a_3x3')(x)
    x = conv2d_bn(x, 80, 1, padding='valid', name='Conv2d_3b_1x1')
    x = conv2d_bn(x, 192, 3, padding='valid', name='Conv2d_4a_3x3')
    x = conv2d_bn(x, 256, 3, strides=2, padding='valid', name='Conv2d_4b_3x3')

    # 5x Block35 (Inception-ResNet-A block):
    for block_idx in range(1, 6):
        x = _inception_resnet_block(x,
                                    scale=0.17,
                                    block_type='Block35',
                                    block_idx=block_idx)

    # Mixed 6a (Reduction-A block):
    channel_axis = 1 if K.image_data_format() == 'channels_first' else 3
    name_fmt = partial(_generate_layer_name, prefix='Mixed_6a')
    branch_0 = conv2d_bn(x,
                         384,
                         3,
                         strides=2,
                         padding='valid',
                         name=name_fmt('Conv2d_1a_3x3', 0))
    branch_1 = conv2d_bn(x, 192, 1, name=name_fmt('Conv2d_0a_1x1', 1))
    branch_1 = conv2d_bn(branch_1, 192, 3, name=name_fmt('Conv2d_0b_3x3', 1))
    branch_1 = conv2d_bn(branch_1,
                         256,
                         3,
                         strides=2,
                         padding='valid',
                         name=name_fmt('Conv2d_1a_3x3', 1))
    branch_pool = MaxPooling2D(3,
                               strides=2,
                               padding='valid',
                               name=name_fmt('MaxPool_1a_3x3', 2))(x)
    branches = [branch_0, branch_1, branch_pool]
    x = Concatenate(axis=channel_axis, name='Mixed_6a')(branches)

    # 10x Block17 (Inception-ResNet-B block):
    for block_idx in range(1, 11):
        x = _inception_resnet_block(x,
                                    scale=0.1,
                                    block_type='Block17',
                                    block_idx=block_idx)

    # Mixed 7a (Reduction-B block): 8 x 8 x 2080
    name_fmt = partial(_generate_layer_name, prefix='Mixed_7a')
    branch_0 = conv2d_bn(x, 256, 1, name=name_fmt('Conv2d_0a_1x1', 0))
    branch_0 = conv2d_bn(branch_0,
                         384,
                         3,
                         strides=2,
                         padding='valid',
                         name=name_fmt('Conv2d_1a_3x3', 0))
    branch_1 = conv2d_bn(x, 256, 1, name=name_fmt('Conv2d_0a_1x1', 1))
    branch_1 = conv2d_bn(branch_1,
                         256,
                         3,
                         strides=2,
                         padding='valid',
                         name=name_fmt('Conv2d_1a_3x3', 1))
    branch_2 = conv2d_bn(x, 256, 1, name=name_fmt('Conv2d_0a_1x1', 2))
    branch_2 = conv2d_bn(branch_2, 256, 3, name=name_fmt('Conv2d_0b_3x3', 2))
    branch_2 = conv2d_bn(branch_2,
                         256,
                         3,
                         strides=2,
                         padding='valid',
                         name=name_fmt('Conv2d_1a_3x3', 2))
    branch_pool = MaxPooling2D(3,
                               strides=2,
                               padding='valid',
                               name=name_fmt('MaxPool_1a_3x3', 3))(x)
    branches = [branch_0, branch_1, branch_2, branch_pool]
    x = Concatenate(axis=channel_axis, name='Mixed_7a')(branches)

    # 5x Block8 (Inception-ResNet-C block):
    for block_idx in range(1, 6):
        x = _inception_resnet_block(x,
                                    scale=0.2,
                                    block_type='Block8',
                                    block_idx=block_idx)
    x = _inception_resnet_block(x,
                                scale=1.,
                                activation=None,
                                block_type='Block8',
                                block_idx=6)

    # Classification block
    x = GlobalAveragePooling2D(name='AvgPool')(x)
    x = Dropout(1.0 - dropout_keep_prob, name='Dropout')(x)
    # Bottleneck
    x = Dense(classes, use_bias=False, name='Bottleneck')(x)
    bn_name = _generate_layer_name('BatchNorm', prefix='Bottleneck')
    x = BatchNormalization(momentum=0.995, epsilon=0.001, scale=False,
                           name=bn_name)(x)

    # Create model
    model = Model(inputs, x, name='inception_resnet_v1')
    if weights_path is not None:
        model.load_weights(weights_path)

    return model

In [ ]:
# Initializing the model.
embeddings_generator = InceptionResNetV1(
        input_shape=(None, None, 3),
        classes=128,
    )
# Loading the prebuilt weights.
embeddings_generator.load_weights('/kaggle/input/indian-actor-faces-for-face-recognition/facenet_keras_weights.h5')

<center> <a href="#top" role="button" aria-pressed="true" >⬆️Back to the Top ⬆️</a>

<a id="2"></a>
<div style="text-align: center; background: #1ED760; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 15px; font-size: 26px; font-weight: bold; line-height: 1; border-radius: 50% 0 50% 0 / 40px; margin-bottom: 20px; box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);">2. Loading Required packages.</div>

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import cv2
import os
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score
from sklearn.preprocessing import LabelEncoder,Normalizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from IPython.display import FileLink
import joblib

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">
<h3 align="left"><font color='#DEB887'>💡 About Packages: </font></h3>
    
* matplotlib              - To display plots and images inline in the notebook.
* seaborn                 - Fore creating visually appealing plots
* numpy                   - For numerical computations.
* pandas                  - To perform operations on tabular data.
* cv2                     - For operations realted to images.
* os                      - To access local file directory.
* tqdm                    - To display the progress of loops.
* sklearn.metrics         - For evaluating the performance of the model
* sklearn.preprocessing   - For functions that helps in preprocessing the data.
* sklearn.svm             - For making use of Support Vector machine algorithm.
* sklearn.model_selection - For dividing the dataset into train and test
* Ipython.display         - For displaying the links using which the utils can be donwloaded.
* joblib                  - To save models and pickle pipelines for restoring them in deployment environment.
</div>

<center> <a href="#top" role="button" aria-pressed="true" >⬆️Back to the Top ⬆️</a>

<a id="3"></a>
<div style="text-align: center; background: #1ED760; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 15px; font-size: 26px; font-weight: bold; line-height: 1; border-radius: 50% 0 50% 0 / 40px; margin-bottom: 20px; box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);">3. Face Embeddings Generation</div>

In [ ]:
def load_images_and_labels(dir_path,model):
    """
    Parameters
    ----------
    dir_path : base directory path that contains the folders of each actor.
    model : facenet model that is loaded with trained weights. (Transfer learning)
    
    Functionality
    -------------
    1. Function reads all the images in the base directory by traversing recursively across all folders.
    2. All the imagees are standardized and the 128 face embeddings will be generated for all the images.
    
    Output
    ------
    face_pixels : list of numpy arrays where each numpy array represents an image (face).
    face_embeddings : list of embeddings for all the images read.
    labels : list of labels where each label represents the name of the actor
    """
    face_pixels,face_embeddings,labels=[],[],[]
    for actor_dir in tqdm(os.listdir(dir_path)):
        for actor_image in os.listdir(dir_path+actor_dir):
            # reading the image
            img=cv2.imread(dir_path+actor_dir+'/'+actor_image)
            
            # converting the image to RGB format
            img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            
            # Resizing the image to the size required by Facenet
            img=cv2.resize(img,(160,160))
            img = img.astype('float32')
            
            # Normalizing the images
            mean, std = img.mean(), img.std()
            img = (img - mean) / std
            face_pixels.append(img)
            labels.append(actor_dir)
            img = np.expand_dims(img, axis=0)
            
            # Getting the face embeddings from the model.
            face_embeddings.append(model.predict(img))
    return face_pixels,face_embeddings,labels

In [ ]:
base_dir='/kaggle/input/indian-actor-faces-for-face-recognition/actors_dataset/Indian_actors_faces/'
face_pixels,face_embeddings,labels=load_images_and_labels(base_dir,embeddings_generator)

In [ ]:
# converting the lists into numpy arrays
face_embeddings=np.array(face_embeddings)
face_pixels=np.array(face_pixels)
labels=np.array(labels)
face_embeddings=face_embeddings.reshape(face_embeddings.shape[0],face_embeddings.shape[2])

### <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#006600; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #003300"> Why is it required to store embeddings?</p>
Facenet takes time for generating embeddings for each images. It takes around 15 miutes to generate face embeddings for the entire dataset. This is a computation intensive task and hence it is always suggested to store the results and reuse them whenever required rather than generating them again.

In [ ]:
# saving these files to overcome processing time of the dataset
np.savez_compressed('/kaggle/working/data.npz',a=face_embeddings,b=face_pixels,c=labels)
FileLink(r'data.npz')

In [ ]:
# loading back the generated embeddings, face images and labels
data=np.load('/kaggle/working/data.npz')
face_embeddings,face_pixels,labels=data['a'],data['b'],data['c']


### <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#006600; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #003300">Checking the authenticity of Embeddings</p>
A small check to make sure that the embeddings generated are proper or not. Found few methods over web that are generating the redundant embeddings for the list of images. They are generating the same embeddings for multiple images. So, we will add a check where we look for unique embeddings. The embeddings of same face will also differ a bit. If we find instances of redundant embeddings, we should make sure that it is corrected before proceeding further.

In [ ]:
# check the authenticity of the generated emebeddings
temp=pd.DataFrame(face_embeddings)
print("Total embeddings generated: ",len(temp))
print("Total embeddings after dropping duplicates: ",len(temp.drop_duplicates()))

Since the number of embeddings did not change even after dropping duplicates, we are moving in correct direction and embeddings generated are valid.

<center> <a href="#top" role="button" aria-pressed="true" >⬆️Back to the Top ⬆️</a>

<a id="4"></a>
<div style="text-align: center; background: #1ED760; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 15px; font-size: 26px; font-weight: bold; line-height: 1; border-radius: 50% 0 50% 0 / 40px; margin-bottom: 20px; box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);">4. Conversion of names of actor faces to discrete labels</div>

In [ ]:
# Fitting a label encoder of names of actors
label_encoder=LabelEncoder()
encoded_labels=label_encoder.fit_transform(labels)

# storing the label encoder object for using in deplyment environment
joblib.dump(label_encoder, 'label_encoder.sav')

# Downloading the picked label encoder file
FileLink(r'label_encoder.sav')

<a id="5"></a>
<div style="text-align: center; background: #1ED760; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 15px; font-size: 26px; font-weight: bold; line-height: 1; border-radius: 50% 0 50% 0 / 40px; margin-bottom: 20px; box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);">5. Splittig the data into train and test sets</div>

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(face_embeddings,encoded_labels,test_size=0.3,stratify=encoded_labels)

In [ ]:
print(f"Shape of Xtrain : {x_train.shape}")
print(f"Shape of Ytrain : {y_train.shape}")
print(f"Shape of Xtest : {x_test.shape}")
print(f"Shape of Ytest : {y_test.shape}")

<center> <a href="#top" role="button" aria-pressed="true" >⬆️Back to the Top ⬆️</a>

<a id="6"></a>
<div style="text-align: center; background: #1ED760; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 15px; font-size: 26px; font-weight: bold; line-height: 1; border-radius: 50% 0 50% 0 / 40px; margin-bottom: 20px; box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);">6. Training a SVM model on data</div>

In [ ]:
model = SVC(probability=True)
model.fit(x_train,y_train )

In [ ]:
yhat_train = model.predict(x_train)
yhat_test = model.predict(x_test)

<a id="7"></a>
<div style="text-align: center; background: #1ED760; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 15px; font-size: 26px; font-weight: bold; line-height: 1; border-radius: 50% 0 50% 0 / 40px; margin-bottom: 20px; box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);">7. Model Evaluation</div>

> <span style='font-size:15px; font-family:Verdana;color: #FF00CC;'><b>7.1 |Accuracy Score</b></span>

In [ ]:
# calculating accuracy score on train set
score_train = accuracy_score(y_train, yhat_train)

# calculating accuracy score on test set
score_test = accuracy_score(y_test, yhat_test)

print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))

> <span style='font-size:15px; font-family:Verdana;color: #FF00CC;'><b>7.2 | Finding the probability for the prediction</b></span>

In [ ]:
# Finding out probabilites of first test sample for every target class
probs_svc = model.decision_function([x_test[0]])
probs_svc = (probs_svc - probs_svc.min()) / (probs_svc.max() - probs_svc.min())
probs_svc

> <span style='font-size:15px; font-family:Verdana;color: #FF00CC;'><b>7.3 | Classification Report</b></span>

In [ ]:
print(classification_report(yhat_test,y_test,target_names=label_encoder.classes_))

> <span style='font-size:15px; font-family:Verdana;color: #FF00CC;'><b>7.4 | Confusion Matrix</b></span>

In [ ]:
print(confusion_matrix(yhat_test,y_test))

<center> <a href="#top" role="button" aria-pressed="true" >⬆️Back to the Top ⬆️</a>

<a id="8"></a>
<div style="text-align: center; background: #1ED760; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 15px; font-size: 26px; font-weight: bold; line-height: 1; border-radius: 50% 0 50% 0 / 40px; margin-bottom: 20px; box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);">8. Downloaing the trained SVM model.</div>


In [ ]:
joblib.dump(model, 'svm_classifier.sav')
FileLink(r'svm_classifier.sav')

<a id="9"></a>
<div style="text-align: center; background: #1ED760; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 15px; font-size: 26px; font-weight: bold; line-height: 1; border-radius: 50% 0 50% 0 / 40px; margin-bottom: 20px; box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);">9. Model Inferencing.</div>

You can test the model with any image in the dataset. To test the model on unseen image, we need to deploy it which will be done later. Now, everytime you execute the below cell, you will get a new image displayed. The title of the image contains the actual name of the actor and the X label of the image contains the predicted name of the actor in the image.

In [ ]:
random_index=np.random.choice(len(labels))
predicted_label=label_encoder.inverse_transform(model.predict([face_embeddings[random_index]]))[0]
actual_label=labels[random_index]
plt.title(f"Actual Label: {actual_label} ")
plt.xlabel(f"Predicted Label: {predicted_label}")
plt.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
plt.imshow(face_pixels[random_index])
plt.show()

<center> <a href="#top" role="button" aria-pressed="true" >⬆️Back to the Top ⬆️</a>

<div style="text-align: center; background:  #FF00CC; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 15px; font-size: 26px; font-weight: bold; line-height: 1; border-radius: 50% 0 50% 0 / 40px; margin-bottom: 20px; box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);">Saving one embedding for each unique face in the dataset</div>


<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">
<h3 align="left"><font color='#DEB887'>💡 Why to store one embedding for every unique face in the dataset?</font></h3>   
This would help in the process of inferencing. After the generation of embeddings, we can recognize the face using either a trained model or using some distances. The embeddings will be in such a way that the distance between two embeddings of the same face will have very less distance when compare to embeddings of different face. This core logic can also be used while inferencing.
</div>

In [ ]:
# Saving one embedding for each face
mapping_dict = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
known_face_embeddings=[]
for key in mapping_dict:
    match_indices=np.where(labels==key)
    first_match_index=match_indices[0][0]
    known_face_embeddings.append(face_embeddings[first_match_index])
known_face_embeddings=np.array(known_face_embeddings)
np.savez_compressed('/kaggle/working/data.npz',a=known_face_embeddings)
FileLink(r'data.npz')

<a id="10"></a>
<div style="text-align: center; background:  #FF00CC; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 15px; font-size: 26px; font-weight: bold; line-height: 1; border-radius: 50% 0 50% 0 / 40px; margin-bottom: 20px; box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);">10. Author Message</div>

<div style="background-color:white;font-size:15px;font-family:Georgia;border-style: solid;border-color: #FF00CC;border-width:3px;padding:10px;margin: 1px;color:#254E58;overflow:hidden"> 

<center><h4><b style = 'color: red;'>Author :</b> Nagasai Biginepalli </h4> </center>
<hr></hr>
<center><b><u>Networking Links</u></b></center>
<b>👉Read more project :</b> https://www.kaggle.com/nagasai524 <br>
<b>👉Shoot me mails :</b> www.biginepallinagasai109@gmail.com<br>
<b>👉Connect on LinkedIn :</b> https://www.linkedin.com/in/nagasai-biginepalli-64648a146/<br>
<b>👉Explore Github :</b> https://github.com/Nagasai524 <br>
    
<hr>
    
<center> <strong> If you liked this Notebook, please do upvote. </strong>
    
<center> <strong> If you have any questions, feel free to comment! </strong>
    
<center> <strong style = 'color: red;' > ✨Best Wishes✨ </strong>

<center> <a href="#top" role="button" aria-pressed="true" >⬆️Back to the Top ⬆️</a>